In [1]:
from utils.gdrive_access import GD2MilvusManager
from tqdm import tqdm
import os
from dotenv import load_dotenv

%reload_ext autoreload
%autoreload 2

/Users/fajrulhaqqi/Public/Career/PHR_ITBA/hackathon/ioc_forum_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

folder_id = os.getenv("GD_FOLDER_ID")   # Replace with actual folder ID
local_folder = "./data"                 # Change to desired local path

# milvus_uri = "http://10.236.124.244:19530"
milvus_uri = "http://localhost:19530"
collection_name = "candidate_colpali"

gdm = GD2MilvusManager(
    milvus_uri=milvus_uri,
    collection_name=collection_name,
    device='mps'
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=248270700232-6220rfekcna210nhtb833b3os62832o6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


Loading checkpoint shards: 100%|██████████| 2/2 [00:39<00:00, 19.61s/it]


In [15]:
gdm.download_files_recursive(folder_id, local_folder, auto_embed=False)

📁 Portfolio
  📄 Ahmad Naufal Rafii.pdf
   ↪ Already Downloaded
  📄 Razan M.pdf
   ↪ Already Downloaded
  📄 Yatin Kholifah.pdf
   ↪ Already Downloaded
  📄 Miki Tjandra A.pdf
   ↪ Already Downloaded
  📄 ENZADA F. PRESTIAWAN.pdf
   ↪ Already Downloaded
  📄 Sarah.pdf
   ↪ Already Downloaded
  📄 Dania Eka Putri.pdf
   ↪ Already Downloaded
  📄 Gustav Anandita.pdf
   ↪ Already Downloaded
  📄 ERIKA ASTRIANA.pdf
   ↪ Already Downloaded
📁 Resume
  📄 Hakinan Toton.pdf
   ↪ Already Downloaded
  📄 Ewangga Arga Wikresa.pdf
   ↪ Already Downloaded
  📄 Nugraha Bayu Syah Putra.pdf
   ↪ Already Downloaded
  📄 Sri Wahyu.pdf
   ↪ Already Downloaded
  📄 Rega Ayundya Putri.pdf
   ↪ Already Downloaded
  📄 Akbar Reza.pdf
   ↪ Already Downloaded
  📄 A Ainun Muthiah Ilyas.pdf
   ↪ Already Downloaded
  📄 A Hafifah.pdf
   ↪ Already Downloaded
  📄 Yuni Lestari .pdf
   ↪ Already Downloaded
  📄 John Philips.pdf
   ↪ Already Downloaded
  📄 Richard Juanda Naibaho.pdf
   ↪ Already Downloaded
  📄 Adrian Hariady Napitupu

In [7]:
filepath = "data/Resume/CV - M Fajrul Haqqi (Feb 25).pdf"
mvc = gdm.milvus
mvc.store_pdf_images_in_milvus(filepath, category="Resume")

 ⦿ Embed image of page 1...
 ⦿ Embed image of page 2...
 ⦿ Stored images in Milvus. ✅
